In [7]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)


In [ ]:
schema = {
  "type": "object",
  "properties": {
    "skills_and_competencies": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "name": {
            "type": "string",
            "description": "The name of the skill or competency identified."
          },
          "type": {
            "type": "string",
            "enum": ["technical", "soft", "domain-specific", "other"],
            "description": "The category of the skill."
          },
          "confidence": {
            "type": "number",
            "minimum": 0,
            "maximum": 1,
            "description": "Confidence score in the inference, from 0 (low) to 1 (high)."
          },
          "evidence": {
            "type": "string",
            "description": "Direct quote or paraphrased section of the interview that supports the inference."
          }
        },
        "required": ["name", "type", "confidence", "evidence"]
      }
    }
  },
  "required": ["skills_and_competencies"]
}

In [16]:
from pydantic import BaseModel, Field
from typing import List, Literal

class BaseSkill(BaseModel):
    pass

class CustomSkill(BaseSkill):
    name: str
    type: Literal["technical", "soft", "domain-specific", "other"]
    confidence: float = Field(ge=0, le=1)
    evidence: str = Field(description="Direct quote or paraphrased section of the interview that supports the inference.")

class SkillList(BaseModel):
    skills: List[BaseSkill]
class CustomSkillList(SkillList):
    skills: List[CustomSkill]


response = client.responses.parse(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": "Extract what skills the user has."},
        {
            "role": "user",
            "content": "I like programming AI and am fairly good at connecting with people.",
        },
    ],
    text_format=CustomSkillList,
)

response.output_parsed

CustomSkillList(skills=[CustomSkill(name='AI Programming', type='technical', confidence=0.9, evidence='I like programming AI'), CustomSkill(name='Interpersonal Skills', type='soft', confidence=0.8, evidence='I am fairly good at connecting with people.')])